In [ ]:
import numpy as np
import json
import os
import cv2
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.semantic_feature_tracker import SemanticFeatureTracker
from lac.slam.frontend import Frontend
from lac.utils.plotting import plot_poses
from lac.util import load_data, compute_odometry_sigmas
from lac.params import DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "/home/shared/data_raw/LAC/runs/triangles_preset6"

initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
left_path = Path(data_path) / "FrontLeft"
right_path = Path(data_path) / "FrontRight"
frames = sorted(int(img_name.split(".")[0]) for img_name in os.listdir(left_path))

In [ ]:
def get_imgs(frame):
    img_name = f"{frame:06}.png"
    left_img = cv2.imread(str(left_path / img_name), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_path / img_name), cv2.IMREAD_GRAYSCALE)
    return {"left": left_img, "right": right_img}

In [ ]:
START_FRAME = 80
END_FRAME = 10000

# TODO: we should probably keyframe the first frame (give it to the backend)

feature_tracker = SemanticFeatureTracker(cam_config)
frontend = Frontend(feature_tracker, initial_pose=initial_pose)

init_data = {
    "step": START_FRAME,
    "FrontLeft": get_imgs(START_FRAME)["left"],
    "FrontRight": get_imgs(START_FRAME)["right"],
    "imu": imu_data[START_FRAME],
}

frontend.initialize(init_data)

In [ ]:
eval_poses = [poses[START_FRAME]]
eval_odoms = []
vo_odoms = []

for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    data = {
        "step": frame,
        "FrontLeft": get_imgs(frame)["left"],
        "FrontRight": get_imgs(frame)["right"],
        "imu": imu_data[frame],
    }
    data = frontend.process_frame(data)
    vo_odoms.append(data["odometry"])
    eval_odoms.append(np.linalg.inv(poses[frame - 2]) @ poses[frame])

In [ ]:
sigma_rotation, sigma_translation = compute_odometry_sigmas(vo_odoms, eval_odoms)
print(f"Rotation sigma (rad): {sigma_rotation}")
print(f"Translation sigma (m): {sigma_translation}")